# Backtest Orbit Model

- [Orbit: A Python Package for Bayesian Forecasting](https://github.com/uber/orbit)
- [Orbit’s Documentation](https://orbit-ml.readthedocs.io/en/stable/index.html)
- [Quick Start](https://orbit-ml.readthedocs.io/en/stable/tutorials/quick_start.html)
- [Orbit: Probabilistic Forecast with Exponential Smoothing](https://arxiv.org/abs/2004.08492) Paper
- [Backtest Orbit Model Documentation](https://orbit-ml.readthedocs.io/en/stable/tutorials/backtest.html)


## Backtest

The way to gauge the performance of a time-series model is through re-training models with different historic periods and check their forecast within certain steps. This is similar to a time-based style cross-validation. More often, we called it `backtest` in time-series modeling.

Two schemes supported for the back-testing engine: **expanding window** and **rolling window**.

### Implemented Models

- ETS (which stands for Error, Trend, and Seasonality) Model
- Methods of Estimations
    - Maximum a Posteriori (MAP)
    - Full Bayesian Estimation
    - Aggregated Posteriors
- Damped Local Trend (DLT)
    - Global Trend Configurations:
        - Linear Global Trend
        - Log-Linear Global Trend
        - Flat Global Trend
        - Logistic Global Trend
    - Damped Local Trend Full Bayesian Estimation (DLTFull)
- Local Global Trend (LGT)
    - Local Global Trend Maximum a Posteriori (LGTMAP)
    - Local Global Trend for full Bayesian prediction (LGTFull)
    - Local Global Trend for aggregated posterior prediction (LGTAggregated)
- Using Pyro for Estimation
    - MAP Fit and Predict
    - VI Fit and Predict
- Kernel-based Time-varying Regression (KTR)
    - Kernel-based Time-varying Regression Lite (KTRLite)

In [ ]:
!pip install orbit-ml==1.0.17 --upgrade --no-input

In [ ]:
import pandas as pd
import numpy as np
import boto3
from sagemaker import get_execution_role
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

import orbit
from orbit.models.dlt import ETSFull, ETSMAP, ETSAggregated, DLTMAP, DLTFull, DLTMAP, DLTAggregated
from orbit.models.lgt import LGTMAP, LGTAggregated, LGTFull
from orbit.models.ktrlite import KTRLiteMAP
from orbit.estimators.pyro_estimator import PyroEstimatorVI, PyroEstimatorMAP
from orbit.diagnostics.backtest import BackTester, TimeSeriesSplitter

import warnings
warnings.filterwarnings('ignore')

In [ ]:
print(orbit.__version__)

In [ ]:
role = get_execution_role()
bucket='...'
data_key = '...'
data_location = 's3://{}/{}'.format(bucket, data_key)

In [ ]:
df = pd.DataFrame(pd.read_csv(data_location))
df = df.drop(df.index[11:]) #drop to forecast for N years only
df['Date'] = pd.to_datetime(df['Date'].astype(str))
df['Value'] = df['Value'].astype(float)

df.head(5)

In [ ]:
date_col = 'Date'
response_col = 'Value'

## Create a TimeSeriesSplitter

### Expanding window

- **expanding window:** for each back-testing model training, the train start date is fixed, while the train end date is extended forward.

In [ ]:
# configs
min_train_len = 4 # minimal length of window length
forecast_len = 2 # length forecast window
incremental_len = 1 # step length for moving forward

In [ ]:
# configs
min_train_len = 5 # in case of rolling window, this specify the length of window length
forecast_len = 1 # length forecast window
incremental_len = 1 # step length for moving forward
window_type = 'expanding' # 'rolling'

In [ ]:
ex_splitter = TimeSeriesSplitter(df,
                                  min_train_len=min_train_len,
                                  incremental_len=incremental_len,
                                  forecast_len=forecast_len,
                                  window_type='expanding', 
                                  date_col='Date')

In [ ]:
print(ex_splitter)

In [ ]:
_ = ex_splitter.plot()

## Create a BackTester

In [ ]:
# instantiate a model
dlt = DLTMAP(
    date_col='Date',
    response_col='Value'
)

lgt_vi = LGTFull(
    response_col='Value',
    date_col='Date',
    seasonality=52,
    seed=8888,
    num_steps=101,
    num_sample=100,
    learning_rate=0.1,
    n_bootstrap_draws=-1,
    estimator_type=PyroEstimatorVI,
)

etsMAP_model = ETSMAP(
        response_col='Value',
        date_col='Date',
        seasonality=52,
        seed=8888,
)

In [ ]:
# configs
min_train_len = 3
forecast_len = 1
incremental_len = 1
window_type = 'expanding'

bt = BackTester(
    model=etsMAP_model,
    df=df,
    min_train_len=min_train_len,
    incremental_len=incremental_len,
    forecast_len=forecast_len,
    window_type=window_type,
)

## Backtest fit and predict

In [ ]:
bt.fit_predict()

In [ ]:
predicted_df = bt.get_predicted_df()
predicted_df.head(50)

In [ ]:
predicted_df.shape

In [ ]:
predicted_df.loc[predicted_df['training_data'] == False]

In [ ]:
help(bt.score)

In [ ]:
score = bt.score()

In [ ]:
def mse_naive(test_actual):
    actual = test_actual[1:]
    predicted = test_actual[:-1]
    return np.mean(np.square(actual - predicted))

def naive_error(train_actual, test_predicted):
    train_mean = np.mean(train_actual)
    return np.mean(np.abs(test_predicted - train_mean))

def rmse(train_actual, test_predicted):
    print(train_actual[-7:])
    print(test_predicted)
    return np.sqrt(np.square(np.subtract(train_actual[-7:],test_predicted[:-1])).mean())

In [ ]:
bt.score(metrics=[rmse])

# 

-------------------------

## Backtesting

### Expanding Window

> for each back-testing model training, the train start date is fixed, while the train end date is extended forward.

### Rolling Window

> for each back-testing model training, the training window length is fixed but the window is moving forward.

In [ ]:
role = get_execution_role()
bucket='...'
data_key = '...' 
data_location = 's3://{}/{}'.format(bucket, data_key)

In [ ]:
df = pd.DataFrame(pd.read_csv(data_location))

In [ ]:
df = df.rename({'Unnamed: 0': 'Date'}, axis = 1)
df.index = df['Date']

In [ ]:
df.shape

In [ ]:
df

#### Orbit Models

In [ ]:
# ETS (which stands for Error, Trend, and Seasonality)

# Methods of Estimations

# Maximum a Posteriori (MAP)

# The advantage of MAP estimation is a faster computational speed.

def ETSMAP_model(date_col, response_col, tmp_df, 
                 min_train_len, forecast_len, incremental_len, window_type):
    
    ets = ETSMAP(
        response_col=response_col,
        date_col=date_col,
        seasonality=52,
        seed=8888
    )
    
    bt = BackTester(
            model=ets,
            df=tmp_df,
            min_train_len=min_train_len,
            incremental_len=incremental_len,
            forecast_len=forecast_len,
            window_type=window_type,
        )
    
    bt.fit_predict()
    
    return bt.score().iloc[5]['metric_values'] # rmsse


# Full Bayesian Estimation


def ETSFull_model(date_col, response_col, tmp_df, 
                 min_train_len, forecast_len, incremental_len, window_type):
    
    ets = ETSFull(
        response_col=response_col,
        date_col=date_col,
        seasonality=52,
        seed=8888,
        num_warmup=400,
        num_sample=400,
    )
    
    bt = BackTester(
            model=ets,
            df=tmp_df,
            min_train_len=min_train_len,
            incremental_len=incremental_len,
            forecast_len=forecast_len,
            window_type=window_type,
        )
    
    bt.fit_predict()
    
    return bt.score().iloc[5]['metric_values'] # rmsse

# Aggregated Posteriors


def ETSAggregated_model(date_col, response_col, tmp_df, 
                        min_train_len, forecast_len, incremental_len, window_type):
    
    ets = ETSAggregated(
        response_col=response_col,
        date_col=date_col,
        seasonality=52,
        seed=8888,
    )
    
    bt = BackTester(
            model=ets,
            df=tmp_df,
            min_train_len=min_train_len,
            incremental_len=incremental_len,
            forecast_len=forecast_len,
            window_type=window_type,
        )
    
    bt.fit_predict()
        
    return bt.score().iloc[5]['metric_values'] # rmsse


# Damped Local Trend (DLT)

# Global Trend Configurations

# Linear Global Trend

# linear global trend
def DLTMAP_lin(date_col, response_col, tmp_df, 
               min_train_len, forecast_len, incremental_len, window_type):
    
    dlt = DLTMAP(
        response_col=response_col,
        date_col=date_col,
        seasonality=52,
        seed=8888,
    )

    bt = BackTester(
            model=dlt,
            df=tmp_df,
            min_train_len=min_train_len,
            incremental_len=incremental_len,
            forecast_len=forecast_len,
            window_type=window_type,
        )
    
    bt.fit_predict()
        
    return bt.score().iloc[5]['metric_values'] # rmsse


# log-linear global trend
def DLTMAP_log_lin(date_col, response_col, tmp_df, 
                   min_train_len, forecast_len, incremental_len, window_type):
    
    dlt = DLTMAP(
        response_col=response_col,
        date_col=date_col,
        seasonality=52,
        seed=8888,
        global_trend_option='loglinear'
    )

    bt = BackTester(
            model=dlt,
            df=tmp_df,
            min_train_len=min_train_len,
            incremental_len=incremental_len,
            forecast_len=forecast_len,
            window_type=window_type,
        )
    
    bt.fit_predict()
        
    return bt.score().iloc[5]['metric_values'] # rmsse


# log-linear global trend
def DLTMAP_flat(date_col, response_col, tmp_df, 
                min_train_len, forecast_len, incremental_len, window_type):
    
    dlt = DLTMAP(
        response_col=response_col,
        date_col=date_col,
        seasonality=52,
        seed=8888,
        global_trend_option='flat'
    )

    bt = BackTester(
            model=dlt,
            df=tmp_df,
            min_train_len=min_train_len,
            incremental_len=incremental_len,
            forecast_len=forecast_len,
            window_type=window_type,
        )
    
    bt.fit_predict()
        
    return bt.score().iloc[5]['metric_values'] # rmsse


# logistic global trend
def DLTMAP_logistic(date_col, response_col, tmp_df, 
                    min_train_len, forecast_len, incremental_len, window_type):
    
    dlt = DLTMAP(
        response_col=response_col,
        date_col=date_col,
        seasonality=52,
        seed=8888,
        global_trend_option='logistic'
    )

    bt = BackTester(
            model=dlt,
            df=tmp_df,
            min_train_len=min_train_len,
            incremental_len=incremental_len,
            forecast_len=forecast_len,
            window_type=window_type,
        )
    
    bt.fit_predict()
    
    return bt.score().iloc[5]['metric_values'] # rmsse


# Damped Local Trend Full Bayesian Estimation (DLTFull)

def DLTFull_model(date_col, response_col, tmp_df, 
                  min_train_len, forecast_len, incremental_len, window_type):
    
    dlt = DLTFull(
        response_col=response_col,
        date_col=date_col,
        num_warmup=400,
        num_sample=400,
        seasonality=52,
        seed=8888
    )
    
    bt = BackTester(
            model=dlt,
            df=tmp_df,
            min_train_len=min_train_len,
            incremental_len=incremental_len,
            forecast_len=forecast_len,
            window_type=window_type,
        )
    
    bt.fit_predict()

    return bt.score().iloc[5]['metric_values'] # rmsse


# Damped Local Trend Full (DLTAggregated)

def DLTAggregated_model(date_col, response_col, tmp_df, 
                        min_train_len, forecast_len, incremental_len, window_type):
    
    ets = DLTAggregated(
        response_col=response_col,
        date_col=date_col,
        seasonality=52,
        seed=8888,
    )
    
    bt = BackTester(
            model=ets,
            df=tmp_df,
            min_train_len=min_train_len,
            incremental_len=incremental_len,
            forecast_len=forecast_len,
            window_type=window_type,
        )
    
    bt.fit_predict()

    return bt.score().iloc[5]['metric_values'] # rmsse


# Local Global Trend (LGT) Model

# Local Global Trend Maximum a Posteriori (LGTMAP)

def LGTMAP_model(date_col, response_col, tmp_df, 
                 min_train_len, forecast_len, incremental_len, window_type):
    
    lgt = LGTMAP(
        response_col=response_col,
        date_col=date_col,
        seasonality=52,
        seed=8888,
    )

    bt = BackTester(
            model=lgt,
            df=tmp_df,
            min_train_len=min_train_len,
            incremental_len=incremental_len,
            forecast_len=forecast_len,
            window_type=window_type,
        )
    
    bt.fit_predict()

    return bt.score().iloc[5]['metric_values'] # rmsse

# LGTFull

def LGTFull_model(date_col, response_col, tmp_df, 
                  min_train_len, forecast_len, incremental_len, window_type):
    
    lgt = LGTFull(
        response_col=response_col,
        date_col=date_col,
        seasonality=52,
        seed=8888,
    )

    bt = BackTester(
            model=lgt,
            df=tmp_df,
            min_train_len=min_train_len,
            incremental_len=incremental_len,
            forecast_len=forecast_len,
            window_type=window_type,
        )
    
    bt.fit_predict()

    return bt.score().iloc[5]['metric_values'] # rmsse

# LGTAggregated

def LGTAggregated_model(date_col, response_col, tmp_df, 
                        min_train_len, forecast_len, incremental_len, window_type):
    
    lgt = LGTAggregated(
        response_col=response_col,
        date_col=date_col,
        seasonality=52,
        seed=8888,
    )

    bt = BackTester(
            model=lgt,
            df=tmp_df,
            min_train_len=min_train_len,
            incremental_len=incremental_len,
            forecast_len=forecast_len,
            window_type=window_type,
        )
    
    bt.fit_predict()

    return bt.score().iloc[5]['metric_values'] # rmsse

# Using Pyro for Estimation

# MAP Fit and Predict

def LGTMAP_PyroEstimatorMAP(date_col, response_col, tmp_df, 
                            min_train_len, forecast_len, incremental_len, window_type):
    
    lgt_map = LGTMAP(
        response_col=response_col,
        date_col=date_col,
        seasonality=52,
        seed=8888,
        estimator_type=PyroEstimatorMAP,
    )

    bt = BackTester(
            model=lgt_map,
            df=tmp_df,
            min_train_len=min_train_len,
            incremental_len=incremental_len,
            forecast_len=forecast_len,
            window_type=window_type,
        )
    
    bt.fit_predict()

    return bt.score().iloc[5]['metric_values'] # rmsse

# VI Fit and Predict

def LGTFull_pyro(date_col, response_col, tmp_df, 
                 min_train_len, forecast_len, incremental_len, window_type):
    
    lgt_vi = LGTFull(
        response_col=response_col,
        date_col=date_col,
        seasonality=52,
        seed=8888,
        num_steps=101,
        num_sample=100,
        learning_rate=0.1,
        n_bootstrap_draws=-1,
        estimator_type=PyroEstimatorVI,
    )

    bt = BackTester(
            model=lgt_vi,
            df=tmp_df,
            min_train_len=min_train_len,
            incremental_len=incremental_len,
            forecast_len=forecast_len,
            window_type=window_type,
        )
    
    bt.fit_predict()

    return bt.score().iloc[5]['metric_values'] # rmsse


# Kernel-based Time-varying Regression (KTR)

# KTRLite

def ktrlite_MAP(date_col, response_col, tmp_df, 
                min_train_len, forecast_len, incremental_len, window_type):
    
    ktrlite = KTRLiteMAP(
        response_col=response_col,
        #response_col=np.log(df[response_col]),
        date_col=date_col,
        level_knot_scale=.1,
        span_level=.05,
    )
    
    bt = BackTester(
            model=ktrlite,
            df=tmp_df,
            min_train_len=min_train_len,
            incremental_len=incremental_len,
            forecast_len=forecast_len,
            window_type=window_type,
        )
    
    bt.fit_predict()

    return bt.score().iloc[5]['metric_values'] # rmsse

#### Backtest all Orbit models for each item

In [ ]:
def backtesing_models(index, column):
    
    '''
    Parameters:
        index: column index
        column: column name
    
    Returns:
        models_df: new dataframe with 
    '''
    
    tmp_df['Date'] = pd.to_datetime(df['Date'].astype(str))
    tmp_df['Penetration'] = df[column].astype(float)
    
    date_col = 'Date'
    response_col = 'Value'

    models_df.at[index ,'Name'] = column

    # configs
    min_train_len = 5 # in case of rolling window, this specify the length of window length
    forecast_len = 1 # length forecast window
    incremental_len = 1 # step length for moving forward
    window_type = 'expanding' # 'rolling' 'expanding'
        
    
    # Making backtesting with each model
    try:
        models_df.at[index , 'ETSMAP'] = ETSMAP_model(date_col, response_col, tmp_df, 
                                                      min_train_len, forecast_len, incremental_len, 
                                                      window_type)
    except:
        models_df.at[index , 'ETSMAP'] = 100
    try:    
        models_df.at[index , 'ETSFull'] = ETSFull_model(date_col, response_col, tmp_df, 
                                                        min_train_len, forecast_len, incremental_len, 
                                                        window_type)
    except:
        models_df.at[index , 'ETSFull'] = 100
    try:
        models_df.at[index , 'ETSAggregated'] = ETSAggregated_model(date_col, response_col, tmp_df, 
                                                        min_train_len, forecast_len, incremental_len, 
                                                        window_type)
    except:
        models_df.at[index , 'ETSAggregated'] = 100

    
    try:
        models_df.at[index , 'DLTMAP_lin'] = DLTMAP_lin(date_col, response_col, tmp_df, 
                                                        min_train_len, forecast_len, incremental_len, 
                                                        window_type)
    except:
        models_df.at[index , 'DLTMAP_lin'] = 100
    try:
        models_df.at[index , 'DLTMAP_log_lin'] = DLTMAP_log_lin(date_col, response_col, tmp_df, 
                                                        min_train_len, forecast_len, incremental_len, 
                                                        window_type)
    except:
        models_df.at[index , 'DLTMAP_log_lin'] = 100
    try:
        models_df.at[index , 'DLTMAP_flat'] = DLTMAP_flat(date_col, response_col, tmp_df, 
                                                        min_train_len, forecast_len, incremental_len, 
                                                        window_type)
    except:
        models_df.at[index , 'DLTMAP_flat'] = 100
    try:
        models_df.at[index , 'DLTMAP_logistic'] = DLTMAP_logistic(date_col, response_col, tmp_df, 
                                                        min_train_len, forecast_len, incremental_len, 
                                                        window_type)
    except:
        models_df.at[index , 'DLTMAP_logistic'] = 100
    try:    
        models_df.at[index , 'DLTFull'] = DLTFull_model(date_col, response_col, tmp_df, 
                                                        min_train_len, forecast_len, incremental_len, 
                                                        window_type)
    except:
        models_df.at[index , 'DLTFull'] = 100
    try:
        models_df.at[index , 'DLTAggregated'] = DLTAggregated_model(date_col, response_col, tmp_df, 
                                                        min_train_len, forecast_len, incremental_len, 
                                                        window_type)
    except:  
        models_df.at[index , 'DLTAggregated'] = 100
    
    
    try:
        models_df.at[index , 'LGTMAP'] = LGTMAP_model(date_col, response_col, tmp_df, 
                                                        min_train_len, forecast_len, incremental_len, 
                                                        window_type)
    except:
        models_df.at[index , 'LGTMAP'] = 100
    try:
        models_df.at[index , 'LGTFull'] = LGTFull_model(date_col, response_col, tmp_df, 
                                                        min_train_len, forecast_len, incremental_len, 
                                                        window_type)
    except: 
        models_df.at[index , 'LGTFull'] = 100
    try: 
        models_df.at[index , 'LGTAggregated'] = LGTAggregated_model(date_col, response_col, tmp_df, 
                                                        min_train_len, forecast_len, incremental_len, 
                                                        window_type)
    except:
        models_df.at[index , 'LGTAggregated'] = 100

    
    # Using Pyro for Estimation
    try:
        models_df.at[index , 'LGTMAP_PyroEstimatorMAP'] = LGTMAP_PyroEstimatorMAP(
            date_col, response_col, tmp_df, min_train_len, forecast_len, incremental_len, window_type)
    except:
        models_df.at[index , 'LGTMAP_PyroEstimatorMAP'] = 100
    try:
        models_df.at[index , 'LGTFull_pyro4'] = LGTFull_pyro(date_col, response_col, tmp_df, 
                                                        min_train_len, forecast_len, incremental_len, 
                                                        window_type)
    except:
         models_df.at[index , 'LGTFull_pyro4'] = 100
        
    # Kernel-based Time-varying Regression (KTR)
    try:
        models_df.at[index , 'KTR_Lite_MAP'] = ktrlite_MAP(date_col, response_col, tmp_df, 
                                                        min_train_len, forecast_len, incremental_len, 
                                                        window_type)
    except:
        models_df.at[index , 'KTR_Lite_MAP'] = 100
    
    
    models_df.at[index, 'Type'] = df[column].iloc[-1]
    
        
    return models_df

#### Calculating minimal RMSSE value for each item

In [ ]:
def min_value(df):
    
    '''
    Parameters:
        df: input dataframe with multiple columns and values in a row
    
    Returns:
        models_df: existing dataframe with added the new 'Model' column filled with 
        the name of the best-fitted model for each item
    '''
        
    df.iloc[:, 1:-1].apply(pd.to_numeric)
    df['Model'] = df.iloc[:, 1:-1].idxmin(axis=1)
    
    return models_df

#### Backtest Orbit models for each item

In [ ]:
import time


tmp_df = pd.DataFrame()
models_df = pd.DataFrame()

#start = time.time()

for index, column in enumerate(df.columns[1:2]):
    evaluating_models(index, column)
    
#end = time.time()
#print(end - start)

In [ ]:
models_df

In [ ]:
min_value(models_df)